In [1]:
import numpy as np
import keras
import pickle
import zarr
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Flatten, BatchNormalization, Activation
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras.layers import *


class RL_Datapoint():
    def __init__(self, state, policy, values):
        self.state = state
        self.policy = policy
        self.values = values
        
def __read_file_data(ID, path='dataset/'):
    dataset = zarr.group(store=zarr.ZipStore(path + str(ID) +'.zip', mode="r"))
    X = np.array(dataset['states'])
    policies = np.array(dataset['policies'])
    values = np.array(dataset['values'])
    return X,policies,values

def __read_files_data(IDs, path='dataset/'):
    X = np.array([])
    for ID in IDs:
        if X.size==0:
            X, policies, values = __read_file_data(ID, path)
            values = np.array(values)
        else:
            X_ID, policies_ID, values_ID = __read_file_data(ID, path)
            X = np.concatenate((X,X_ID))
            policies = np.concatenate((policies, policies_ID))
            values = np.concatenate((values,np.array(values_ID)))
    return X,policies,values
        
    
    
def generator(batch_size, datasetFileIds, path='dataset/'):
    while True:
        np.random.shuffle(datasetFileIds)

        for file_step, file_id in enumerate(datasetFileIds): 
            X, policies, values = __read_file_data(file_id,path)
#             X = np.rollaxis(X, 1,4).shape
            rand_indices = np.arange(len(X))
            np.random.shuffle(rand_indices)

            for i in range(int(len(X)/batch_size)):
                batch_indices = rand_indices[i*batch_size: (i+1)*batch_size]
                yield X[batch_indices],[policies[batch_indices],values[batch_indices]]
                
def validation_generator(batch_size, datasetFileIds, path='dataset/'):
    while True:
        np.random.shuffle(datasetFileIds)

        for file_step, file_id in enumerate(datasetFileIds): 
            X, policies, values = __read_file_data(file_id,path)
#             X = np.rollaxis(X, 1,4).shape
            rand_indices = np.arange(len(X))
            np.random.shuffle(rand_indices)

            for i in range(int(len(X)/batch_size)):
                batch_indices = rand_indices[i*batch_size: (i+1)*batch_size]
                yield X[batch_indices],[policies[batch_indices],values[batch_indices]]
                
def plotHistory( history):
    import matplotlib.pyplot as plt

    val_loss = history.history['val_loss']
    val_policy_loss = history.history['val_policy_loss']
    val_value_loss = history.history['val_value_loss']
    loss = history.history['loss']
    policy_loss = history.history['policy_loss']
    value_loss = history.history['value_loss']
    
    epochs = range(1,len(loss) + 1)

    # fig, ax = plt.subplots(nrows=2, ncols=2)
    fig = plt.figure()

    plt.subplot(2, 2, 1)
    plt.plot(epochs,loss,'bo',label='loss')
    plt.plot(epochs,val_loss,'b',label='val_loss')
    plt.title = 'Training and validation loss'
    plt.legend()

    plt.subplot(2, 2, 2)
    plt.plot(epochs,policy_loss,'bo',label='policy_loss')
    plt.plot(epochs,val_policy_loss,'b',label='val_policy_loss')
    plt.title = 'Training and validation policy loss'
    plt.legend()        

    plt.subplot(2, 2, 3)
    plt.plot(epochs,value_loss,'bo',label='value_loss')
    plt.plot(epochs,val_value_loss,'b',label='val_value_loss')
    plt.title = 'Training and validation value loss'
    plt.legend()
    plt.show()
    
def acc_sign(y_true, y_pred):
    return K.mean(K.equal(K.sign(y_pred), K.sign(y_true)), axis=-1)


def acc_round(y_true, y_pred):
    # each interval for class -1, 0, 1 must have a width of 2/3 to be equally distributed
    # therefore you must convert (1/3) to (1/2) and only then round to the closest integer
    # 1/3 * x = 1/2
    return K.mean(K.equal(K.round(y_pred * 1.5), K.round(y_true)), axis=-1)

def acc_reg(y_true,y_pred):
    return K.constant(1) - K.square(K.mean((y_pred-y_true), axis=1))

def acc_round_unequal(y_true,y_pred):
    return K.mean(K.equal(K.round(y_true),K.round(y_pred)), axis=-1)

Using TensorFlow backend.


In [87]:
x, p, v = __read_files_data([0,1],dataset_path)
print(x.shape, p.shape, v.shape)
print(x.nbytes/1024/1024,'MB')

(20000, 60, 8, 8) (20000, 2272) (20000, 1)
585.9375 MB


In [2]:
def getResidualNetwork(input_shape, output_value=1, output_policy=2272):
    
    channel_pos = 'channels_first'
    inp_shape = Input(input_shape,name='input1')
    x = Conv2D(256, kernel_size=(3,3), padding = 'same', input_shape=input_shape,data_format=channel_pos,name='conv2d_Prep')(inp_shape)
    x = BatchNormalization(axis=1,name='batch_normalization_prep')(x)
    x_a1 = Activation('relu',name='activation_prep')(x)
    activated_x = x_a1
    
#     activated_x, x
    def createResidualBlock(block_nr, activated_x):
        nr = block_nr *2 -1
        x = Conv2D(256, kernel_size=(3,3), name = 'conv2d_'+str(nr), padding='same',data_format=channel_pos)(activated_x)
        x = BatchNormalization(axis=1, name = 'batch_normalization_'+str(nr))(x)
        x = Activation('relu',name = 'activation_'+str(nr))(x)
        x = Conv2D(256, kernel_size=(3,3), name = 'conv2d_'+str(nr+1),padding = 'same',data_format=channel_pos)(x)
        x = BatchNormalization(axis=1, name = 'batch_normalization_'+str(nr+1))(x)
        x = keras.layers.add([x,activated_x],name='add_' + str(block_nr))
        activated_x = Activation('relu',name='activation_'+str(nr+1))(x)
        return activated_x
    
    # build eight residual blocks
    for i in range (1,8):
        activated_x = createResidualBlock(i, activated_x)
        

    
    # Value header
    x = Conv2D(4, kernel_size=(1,1),name='value_conv2d', padding = 'same',data_format=channel_pos)(activated_x)
    xb = BatchNormalization(axis=1,name='value_batch_normalization')(x)
    xA = Activation('relu',name='value_activation')(xb)
    xF = Flatten(name='value_flatten')(xA)
    dense_1 = Dense(256, activation='relu',name='value_dense')(xF)
    value = Dense(output_value, activation='tanh', name='value')(dense_1)

    # Policy Header
    xConv = Conv2D(8, kernel_size=(7,7), padding = 'same',name='policy_conv2d',data_format=channel_pos)(activated_x)
    xb = BatchNormalization(axis=1,name='policy_batch_normalization')(xConv)
    xA = Activation('relu',name='policy_activation')(xb)
    xF = Flatten(name='policy_flatten')(xA)
    policy = Dense(output_policy, activation='softmax', name='policy')(xF)


    from keras.models import Model
    model = Model(inp_shape, [policy,value])

    model.summary()
    return model


<h3>NN Architecture: ResNet</h3>

<h3>Main Parameters of the NN</h3>

In [3]:
# ----------- parameters ----------------
files_len = 1000                                # amount of zip files
file_id_start = 0
file_ids = np.arange(file_id_start,files_len)               
# np.random.shuffle(file_ids)
train_ids = file_ids[:int(len(file_ids)*0.9)] # file_ids for training set
val_ids = file_ids[int(len(file_ids)*0.9):]   # file_ids for validation set 
filepath="models/model-{epoch:02d}.hdf5"      # path to save model
zip_length = 10000                            # amount of datapoints in a zip file
data_len = files_len * zip_length             # whole amount of datapoints in all zip files
batch_size = 256                              # batch size
steps_per_epoch = int((len(train_ids)*zip_length)/batch_size) # amount of batches in one epoch
CLASSES_LEN = 2272                            # amount of classes for policy
channel_pos = 'channels_last'          
dataset_path = 'dataset/'                     # relative directory to the dataset
inp_shape = (60,8,8)                          # shape of a datapoint
val_id = files_len -1 
x_val, policies_val, values_val = __read_file_data(6000,path=dataset_path)
print('train_ids:',np.min(train_ids),' - ', np.max(train_ids))
print('val_ids:',np.min(val_ids),' - ', np.max(val_ids))
print('x_val:', x_val.shape)

train_ids: 0  -  899
val_ids: 900  -  999
x_val: (10000, 60, 8, 8)


In [59]:
np.concatenate((x_val,x_val)).shape

(20000, 60, 8, 8)

In [97]:
def evaluate_model(model, file_ids, dataset_path):
    x_val, policies_val, values_val = __read_files_data(file_ids,path=dataset_path)
    print(model.metrics_names)
    print(model.evaluate(x_val,[policies_val,values_val]))

evaluate_model(model,[6001,6002,6003],dataset_path)

['loss', 'policy_loss', 'value_loss', 'policy_acc', 'value_acc_round', 'value_acc_sign']
30000/30000 [==============================] - 49s 2ms/step
[1.095079911295573, 1.8439676438967387, 0.8454506663044293, 0.5189, 0.23126666666666668, 0.5441666666666667]


In [94]:
x_val, policies_val, values_val = __read_files_data([6001],path=dataset_path)
print(x_val.shape,policies_val.shape)

(10000, 60, 8, 8) (10000, 2272)


<h3>Build the model</h3>

In [4]:
model = getResidualNetwork(inp_shape, output_policy=CLASSES_LEN)
sgd = optimizers.SGD(lr=0.000, momentum=0.9, decay=0.0, nesterov=False)

model.compile(loss={'policy':'categorical_crossentropy',
                    'value':'mean_squared_error'}, optimizer=sgd,
              metrics={'policy':'accuracy', 'value':[acc_round, acc_sign]}, loss_weights=[0.25,0.75])

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input1 (InputLayer)             (None, 60, 8, 8)     0                                            
__________________________________________________________________________________________________
conv2d_Prep (Conv2D)            (None, 256, 8, 8)    138496      input1[0][0]                     
__________________________________________________________________________________________________
batch_normalization_prep (Batch (None, 256, 8, 8)    1024        conv2d_Prep[0][0]                
__________________________________________________________________________________________________
activation_prep (Activation)    (None, 256, 8, 8)    0           batch_normalization_prep[0][0]   
_____________________________________

In [5]:
model.load_weights('finalModel/model-05.hdf5')

In [6]:
model.evaluate(x_val,[policies_val,values_val],verbose=True)

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_Prep/convolution}}]]
	 [[{{node loss/add}}]]

In [98]:
x_val[0].shape

(60, 8, 8)

In [102]:
np.expand_dims(x_val[0],0).shape

(1, 60, 8, 8)

In [108]:
board = x_val[0][np.newaxis, :, :]
board.shape

(1, 60, 8, 8)

In [7]:
data = np.load('data.npy')

In [8]:
model.predict(data)

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_Prep/convolution}}]]
	 [[{{node policy/Softmax}}]]

In [107]:
model.predict(x_test)

[array([[8.0579248e-08, 1.4189803e-07, 1.6486984e-08, ..., 1.7541266e-08,
         9.9875290e-09, 1.2479663e-08]], dtype=float32),
 array([[0.07075516]], dtype=float32)]

<h3>Define callbacks</h3>

<h4>Callback for checkpoint and tensorboard</h4>

In [34]:
# callbacks 
from datetime import datetime
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
logdir="logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir, update_freq='batch')

<h4>Callback for Learning rate</h4>

In [35]:
# learning rate
from  LearningRateScheduler import *
epochs = 5
batch_len = epochs * int(data_len/ (batch_size))
max_lr = 0.001*8
total_it = batch_len
min_lr = 0.0001
print('BatchLen: ', batch_len, ' - DataLen: ', data_len)
lr_schedule = OneCycleSchedule(start_lr=max_lr/8, max_lr=max_lr, cycle_length=total_it*.4, cooldown_length=total_it*.6, finish_lr=min_lr)
scheduler = LinearWarmUp(lr_schedule, start_lr=min_lr, length=total_it/30)
bt = BatchLearningRateScheduler(scheduler)

BatchLen:  195310  - DataLen:  10000000


In [36]:
callbacks_list = [checkpoint,tensorboard_callback,bt]
# callbacks_list = [checkpoint,bt]


In [37]:
from keras import backend
backend.set_image_dim_ordering('th')
print(backend.image_data_format())

channels_first


In [38]:
# model.save('models\\BughouseNet220620190437.h5')
history = model.fit_generator(generator(batch_size,train_ids,path=dataset_path), 
                                steps_per_epoch=int((len(train_ids)*zip_length)/batch_size), callbacks=callbacks_list,
                                epochs=epochs, validation_data=(x_val, [policies_val,values_val]))


Epoch 1/5
    3/35156 [..............................] - ETA: 10:55:13 - loss: 2.7828 - policy_loss: 7.8073 - value_loss: 1.1080 - policy_acc: 0.0000e+00 - value_acc_round: 0.3138 - value_acc_sign: 0.5234

C:\Users\MoBray\.conda\envs\TensorFlow_GPU_Keras\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.308673). Check your callbacks.
  % delta_t_median)


35156/35156 [==============================] - 6442s 183ms/step - loss: 1.5378 - policy_loss: 3.8857 - value_loss: 0.7552 - policy_acc: 0.3314 - value_acc_round: 0.1492 - value_acc_sign: 0.5962 - val_loss: 1.3462 - val_policy_loss: 2.9029 - val_value_loss: 0.8274 - val_policy_acc: 0.4004 - val_value_acc_round: 0.1298 - val_value_acc_sign: 0.5418alue_loss: 0.7552 - policy_acc: 0.3314 - value_acc_round: 0

Epoch 00001: val_loss improved from inf to 1.34624, saving model to models/model-01.hdf5
Epoch:  1  - lr: 0.006133369  - batch: 0  - epoch:  1
Epoch 2/5
35156/35156 [==============================] - 9879s 281ms/step - loss: 1.0493 - policy_loss: 2.0495 - value_loss: 0.7160 - policy_acc: 0.5116 - value_acc_round: 0.2315 - value_acc_sign: 0.6281 - val_loss: 1.1435 - val_policy_loss: 1.9284 - val_value_loss: 0.8819 - val_policy_acc: 0.5232 - val_value_acc_round: 0.2163 - val_value_acc_sign: 0.5354ss: 2.0504 - value_loss: 0.7160 - policy_ac - ETA: 18s - loss: 1.0495 - policy_loss:

Epoch 

In [39]:
model.save('models\\FinalModel05072019.h5')


In [ ]:
# model.save('models\\BughouseNet220620190437.h5')
history = model.fit_generator(generator(batch_size,train_ids,path=dataset_path), steps_per_epoch=int((len(train_ids)*zip_length)/batch_size), callbacks=callbacks_list,
                    epochs=epochs, validation_data=(x_val, [policies_val,values_val]))


Instructions for updating:
Use tf.cast instead.
Epoch 1/5
    3/35156 [..............................] - ETA: 11:05:33 - loss: 2.6790 - policy_loss: 8.3141 - value_loss: 0.8006 - policy_acc: 0.0052 - value_acc_round: 0.0951

e:\python\v37\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.184831). Check your callbacks.
  % delta_t_median)


    4/35156 [..............................] - ETA: 9:23:09 - loss: 2.6764 - policy_loss: 8.3132 - value_loss: 0.7974 - policy_acc: 0.0059 - value_acc_round: 0.0957 

e:\python\v37\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.265181). Check your callbacks.
  % delta_t_median)


    5/35156 [..............................] - ETA: 7:55:19 - loss: 2.6645 - policy_loss: 8.2937 - value_loss: 0.7881 - policy_acc: 0.0055 - value_acc_round: 0.1055

...

22377/35156 [==================>...........] - ETA: 49:24 - loss: 0.8292 - policy_loss: 1.2694 - value_loss: 0.6825 - policy_acc: 0.6363 - value_acc_round: 0.1797

e:\python\v37\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.232308). Check your callbacks.
  % delta_t_median)


22378/35156 [==================>...........] - ETA: 49:24 - loss: 0.8292 - policy_loss: 1.2694 - value_loss: 0.6825 - policy_acc: 0.6363 - value_acc_round: 0.1797

e:\python\v37\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.235257). Check your callbacks.
  % delta_t_median)


22751/35156 [==================>...........] - ETA: 47:53 - loss: 0.8292 - policy_loss: 1.2684 - value_loss: 0.6829 - policy_acc: 0.6365 - value_acc_round: 0.1793

In [ ]:
# model.save('models\\BughouseNet220620190437.h5')
history = model.fit_generator(generator(batch_size,len(train_ids),path=dataset_path), steps_per_epoch=int((len(train_ids)*zip_length)/batch_size), callbacks=callbacks_list,
                    epochs=epochs, validation_data=(x_val, [policies_val,values_val]))


In [ ]:
# model.save('models\\BughouseNet220620190437.h5')
history = model.fit_generator(generator(batch_size,len(train_ids),path=dataset_path), steps_per_epoch=int((len(train_ids)*zip_length)/batch_size), callbacks=callbacks_list,
                    epochs=epochs, validation_data=(x_val, [policies_val,values_val]))


In [ ]:
# model.save('models\\BughouseNet220620190437.h5')
history = model.fit_generator(generator(batch_size,len(train_ids),path=dataset_path), steps_per_epoch=int((len(train_ids)*zip_length)/batch_size), callbacks=callbacks_list,
                    epochs=epochs, validation_data=(x_val, [policies_val,values_val]))


In [ ]:
# model.save('models\\BughouseNet220620190437.h5')
history = model.fit_generator(generator(batch_size,len(train_ids),path=dataset_path), steps_per_epoch=int((len(train_ids)*zip_length)/batch_size), callbacks=callbacks_list,
                    epochs=epochs, validation_data=(x_val, [policies_val,values_val]))


In [ ]:
# model.save('models\\BughouseNet220620190437.h5')
history = model.fit_generator(generator(batch_size,len(train_ids),path=dataset_path), steps_per_epoch=int((len(train_ids)*zip_length)/batch_size), callbacks=callbacks_list,
                    epochs=epochs, validation_data=(x_val, [policies_val,values_val]))


In [ ]:
# model.save('models\\BughouseNet220620190437.h5')
history = model.fit_generator(generator(batch_size,len(train_ids),path=dataset_path), steps_per_epoch=int((len(train_ids)*zip_length)/batch_size), callbacks=callbacks_list,
                    epochs=epochs, validation_data=(x_val, [policies_val,values_val]))


In [ ]:
plotHistory(history)


<h3>MNist overfitting</h3>

In [ ]:
def getResidualNetwork(input_shape):
    
    channel_pos = 'channels_first'
    inp_shape = Input(input_shape,name='input1')
    x = Conv2D(256, kernel_size=(3,3), padding = 'same', input_shape=input_shape,data_format=channel_pos,name='conv2d_1')(inp_shape)
    x = BatchNormalization(axis=1,name='batch_normalization_1')(x)
    x_a1 = Activation('relu',name='activation_1')(x)

    x = Conv2D(256, kernel_size=(3,3),name ='conv2d_2' ,padding = 'same',data_format=channel_pos)(x_a1)
    x = BatchNormalization(axis=1, name = 'batch_normalization_2')(x)
    x = Activation('relu',name = 'activation_2')(x)
    x = Conv2D(256, kernel_size=(3,3), name = 'conv2d_3',padding = 'same',data_format=channel_pos)(x)
    x = BatchNormalization(axis=1, name = 'batch_normalization_3')(x)

    x = keras.layers.add([x,x_a1],name='add1')
    x_a2 = Activation('relu',name='activation_3')(x)
    x = Conv2D(256, kernel_size=(3,3),name = 'conv2d_4', padding = 'same',data_format=channel_pos)(x_a2)
    x = BatchNormalization(axis=1,name = 'batch_normalization_4')(x)
    x = Activation('relu',name='activation_4')(x)
    x = Conv2D(256, kernel_size=(3,3), name='conv2d_5',padding = 'same',data_format=channel_pos)(x)
    x = BatchNormalization(axis=1,name='batch_normalization_5')(x)

    x = keras.layers.add([x,x_a2],name='add_2')
    x_a3 = Activation('relu',name='activation_5')(x)
    x = Conv2D(256, kernel_size=(3,3),name='conv2d_6', padding = 'same',data_format=channel_pos)(x_a3)
    x = BatchNormalization(axis=1,name='batch_normalization_6')(x)
    x = Activation('relu',name='activation_6')(x)
    x = Conv2D(256, kernel_size=(3,3), name='conv2d_7',padding = 'same',data_format=channel_pos)(x)
    x = BatchNormalization(axis=1,name='batch_normalization_7')(x)

    x = keras.layers.add([x,x_a3],name='add_3')
    x_a4 = Activation('relu',name='activation_7')(x)
    x = Conv2D(256, kernel_size=(3,3), name = 'conv2d_8',padding = 'same',data_format=channel_pos)(x_a4)
    x = BatchNormalization(axis=1,name='batch_normalization_8')(x)
    x = Activation('relu',name='activation_8')(x)
    x = Conv2D(256, kernel_size=(3,3), name='conv2d_9',padding = 'same',data_format=channel_pos)(x)
    x = BatchNormalization(axis=1,name='batch_normalization_9')(x)

    x = keras.layers.add([x,x_a4],name='add4')
    x_a5 = Activation('relu',name='activation_9')(x)
    x = Conv2D(256, kernel_size=(3,3),name='conv2d_10', padding = 'same',data_format=channel_pos)(x_a5)
    x = BatchNormalization(axis=1,name='batch_normalization_10')(x)
    x = Activation('relu',name='activation_10')(x)
    x = Conv2D(256, kernel_size=(3,3),name='conv2d_11', padding = 'same',data_format=channel_pos)(x)
    x = BatchNormalization(axis=1,name='batch_normalization_11')(x)

    x = keras.layers.add([x,x_a5],name='add_5')
    x_a6 = Activation('relu',name='activation_11')(x)
    x = Conv2D(256, kernel_size=(3,3), name='conv2d_12',padding = 'same',data_format=channel_pos)(x_a6)
    x = BatchNormalization(axis=1,name='batch_normalization_12')(x)
    x = Activation('relu',name='activation_12')(x)
    x = Conv2D(256, kernel_size=(3,3), name='conv2d_13',padding = 'same',data_format=channel_pos)(x)
    x = BatchNormalization(axis=1,name='batch_normalization_13')(x)

    x = keras.layers.add([x,x_a6],name='add6')
    x_a7 = Activation('relu',name='activation_13')(x)
    x = Conv2D(256, kernel_size=(3,3), name='conv2d_14',padding = 'same',data_format=channel_pos)(x_a7)
    x = BatchNormalization(axis=1,name='batch_normalization_14')(x)
    x = Activation('relu',name='activation_14')(x)
    x = Conv2D(256, kernel_size=(3,3), name='conv2d_15',padding = 'same',data_format=channel_pos)(x)
    x = BatchNormalization(axis=1,name='batch_normalization_15')(x)

    x = keras.layers.add([x,x_a7],name='add_7')
    x_a8 = Activation('relu',name='activation_15')(x)
    x = Conv2D(1, kernel_size=(1,1),name='conv2d_17', padding = 'same',data_format=channel_pos)(x_a8)
    xb = BatchNormalization(axis=1,name='batch_normalization_17')(x)
    xConv = Conv2D(8, kernel_size=(7,7), padding = 'same',name='conv2d_16',data_format=channel_pos)(x_a8)
    xA = Activation('relu',name='activation_17')(xb)
    xb = BatchNormalization(axis=1,name='batch_normalization_16')(xConv)
    xF = Flatten(name='flatten_2')(xA)
    xA = Activation('relu',name='activation_16')(xb)

    dense_1 = Dense(256, activation='relu',name='dense_1')(xF)
    xF = Flatten(name='flatten_1')(xA)

    value = Dense(1, activation='tanh', name='value')(dense_1)
    policy = Dense(10, activation='softmax', name='policy')(xF)

    from keras.models import Model
    model = Model(inp_shape, [policy])

    model.summary()
    return model

In [ ]:
from keras.datasets import mnist
img_rows, img_cols = 28, 28
num_classes = 10
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
from  LearningRateScheduler import *
data_len = len(x_train)
epochs = 20
batch_len = epochs * int(data_len/ (batch_size))
max_lr = 0.001*8
total_it = batch_len
min_lr = 0.0001
print('BatchLen: ', batch_len, ' - DataLen: ', data_len)
lr_schedule = OneCycleSchedule(start_lr=max_lr/8, max_lr=max_lr, cycle_length=total_it*.4, cooldown_length=total_it*.6, finish_lr=min_lr)
scheduler = LinearWarmUp(lr_schedule, start_lr=min_lr, length=total_it/30)
bt = BatchLearningRateScheduler(scheduler)

In [ ]:
model = getResidualNetwork(input_shape)
sgd = optimizers.SGD(lr=0.000, momentum=0.9, decay=0.0, nesterov=False)

def acc_reg(y_true,y_pred):
    return K.constant(1) - K.square(K.mean((y_pred-y_true), axis=1))
callbacks_list = [bt]

model.compile(loss=['categorical_crossentropy'], optimizer=sgd,
              metrics=['accuracy'])
# model.fit(x_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(x_test, y_test),
#          callbacks=callbacks_list)
